<a href="https://colab.research.google.com/github/DebbieMatt/PYTHON_DW/blob/main/Integra%C3%A7%C3%A3o_Dados_IBGE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import pandas as pd
from datetime import datetime
from pathlib import Path
import sqlite3 as sql


In [40]:
dtCarga = datetime.today().strftime('%d/%m/%Y %H:%M')

# COLETA DE DADOS DO SITE IBGE

In [22]:
# Coletando dados do IBGE

url = 'https://www.ibge.gov.br/explica/codigos-dos-municipios.php#MT'

DadosIBGE = pd.DataFrame (pd.read_html(url, match= "Municípios de Mato Grosso")[0])


In [23]:
# RENOMEAR COLUNA DO DF

DadosIBGE = DadosIBGE.rename(columns={'Municípios de Mato Grosso': 'Munic', 'Códigos': 'Cod'} )

# ALTERANDO NOME DO INDEX
DadosIBGE.index.name = 'ID'

In [24]:
# ALTERANDO INDEX PARA COMEÇAR DO 1
DadosIBGE.index = DadosIBGE.index + 1

In [41]:
# ADICIONAR UMA COLUNA DE DATA E HORA DE CARGA
DadosIBGE['dataCarga'] = dtCarga

# DadosIBGE

# CRIANDO O BANCO DE DADOS ODS

In [26]:
# Manipulando o sistema de arquivos
endereco = Path('/content/drive/MyDrive/Colab Notebooks')

BDODS = endereco / "ODS.db"
BDDW = endereco / "DW.db"



In [27]:
if endereco.exists():
    if(BDODS.exists() and BDDW.exists()):
        print('Banco de dados já existem !')
    else:
        BDODS.touch()
        BDDW.touch()
        print('Bancos de dados criados!')
else:
    print('Endereço não existe! Favor, verificar!')


Banco de dados já existem !


# MANIPULANDO OS BANCO DE DADOS CRIADOS


In [28]:
# Conectar no BDODS
conexaoODS = sql.connect(BDODS)

# Criar a tabela tbLogMunic e carregar os dados do DF DadosIBGE
DadosIBGE.to_sql('tabelaLogMunic', conexaoODS, if_exists="append")

#Confirmar a transação
conexaoODS.commit()

#Fechar a conexão
conexaoODS.close()

print('Carga do BDODS concluída!')

Carga do BDODS concluída!


In [29]:
#Conectar no BDDW
conexaoDW = sql.connect(BDDW)

#Selecionar somente as colunas para criação da dMunicipio
DadosIBGE = DadosIBGE[['Munic','Cod']]

#Criar a tabela dMunicipio e carregar os dados do DF DadosIBGE
DadosIBGE.to_sql('dMunicipio',conexaoDW,if_exists="replace")

#Confirmar a transação
conexaoDW.commit()

#Fechar a conexão
conexaoDW.close()

print('Carga do DW concluída!')


Carga do DW concluída!



# CARGA DOS DADOS DO PIB DE SETORES DE MATO GROSSO

In [34]:
#Definir o endereço físico do arquivo CSV
LocalArquivo = "/content/drive/MyDrive/Colab Notebooks/"

In [39]:
#Criar Data Frame com os dados dos PIB - SETOR - MT
tbLogPIBsetor = pd.DataFrame(pd.read_csv(LocalArquivo + "PIB-SETOR-MT.csv"))

In [44]:
#Criando coluna com a data da carga
tbLogPIBsetor['dtCarga'] = dtCarga


# CARGA DOS DADOS DOS IMPOSTOS SOBRE O  PIB DOS SETORES DO ESTADO DE MATO GROSSO

In [47]:
#Criar Data Frame com os dados dos PIB - MT
tbLogPIBmt = pd.DataFrame(pd.read_csv(LocalArquivo + "PIB-MT.csv"))

In [50]:
#Criando coluna com a data da carga
tbLogPIBmt['dtCarga'] = dtCarga
